In [68]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

### Import

In [69]:
import json

### Get recipes data

In [70]:
f = open('/kaggle/input/food-recipes-dataset/recipes.json')
recipes_data = json.load(f)
recipes = recipes_data['recipes']
# recipes

In [71]:
recipes_df = pd.DataFrame(recipes)

### Get users data 

In [72]:
f = open('/kaggle/input/food-recipes-dataset/users.json')
users_data = json.load(f)
users = users_data['users']
# users

In [73]:
users_df = pd.DataFrame(users)
users_df = users_df[['id', 'userName']]
users_df['recipes_id'] = pd.np.empty((len(users_df), 0)).tolist()
users_df = users_df.set_index('id')
# users_df

### Init recipes nutrition 

In [74]:
recipes_df['Total Fat'] = '0'
recipes_df['Saturated Fat'] = '0'
recipes_df['Sugars'] = '0'
recipes_df['Sodium'] = '0'
recipes_df['Protein'] = '0'

### Get nutrition of recipes and get recipes id of users

In [75]:
for i in range(len(recipes_df)):
    recipes_df.at[i,'Sodium'] = recipes_df.loc[i].nutrition[4]['quantity']
    recipes_df.at[i,'Total Fat'] = recipes_df.loc[i].nutrition[1]['quantity']
    recipes_df.at[i,'Saturated Fat'] = recipes_df.loc[i].nutrition[2]['quantity']
    recipes_df.at[i,'Sugars'] = recipes_df.loc[i].nutrition[3]['quantity'] 
    recipes_df.at[i,'Protein'] = recipes_df.loc[i].nutrition[5]['quantity']

    users_df.loc[recipes_df.loc[i]['authorId']]['recipes_id'].append(recipes_df.loc[i]['id'])

### Remove columns recipes df

In [76]:
recipes_df = recipes_df[['id', 'category', 'Total Fat', 'Saturated Fat', 'Sugars', 'Sodium', 'Protein']]

### Convert float type

In [77]:
recipes_df[['Total Fat', 'Saturated Fat', 'Sugars', 'Sodium', 'Protein']] = recipes_df[['Total Fat', 'Saturated Fat', 'Sugars', 'Sodium', 'Protein']].astype('float')

In [78]:
users_df.head()

### Create matrix 

In [79]:
cal = recipes_df.pivot_table(columns='id',values=['Total Fat', 'Saturated Fat', 'Sugars', 
                                                  'Sodium', 'Protein'])
cal

### Predict function

In [80]:

def predict(recipe_name, recipe_number = 5):
    nutrition_recipe = cal[recipe_name]
    
    # find similar recipes
    similar = cal.corrwith(nutrition_recipe)
    
    corr = pd.DataFrame(similar, columns=['Correlation'])
    corr = corr[corr['Correlation'] > 0]
    corr.dropna(inplace=True)
    
    # sort by correlation
    corr = corr.sort_values('Correlation', ascending=False)
    return corr.iloc[1:recipe_number + 1, :].index.to_list()


### Predict each recipe

In [81]:
recipes_df['recipes_red'] = recipes_df['id'].apply(lambda x : predict(x, 3))

In [82]:
recipes_df

### Evaluation

In [83]:
def predict_mul(arr):
    init_list = []
    for i in range(len(arr)):
        init_list = list(set(init_list + predict(arr[i])))
    return init_list

In [84]:
users_df['pred_given_1'] = users_df['recipes_id'].apply(lambda x : predict(x[0]))

In [85]:
users_df['pred_given_2'] = users_df['recipes_id'].apply(lambda x : predict_mul(x[:2]))

In [86]:
users_df['pred_but_at_all'] = users_df['recipes_id'].apply(lambda x : predict_mul(x[1:]))

In [87]:
def is_slice_in_list(s,l):
    for i in range(len(l)):
        if l[i] in s:
            return True
        else:
            continue
    return False

In [88]:
users_df['result_given_1'] = False
users_df['result_given_2'] = False
users_df['result_but_at_all'] = False

In [89]:
for idx in users_df.index:
    if(is_slice_in_list(users_df.loc[idx]['pred_given_1'], users_df.loc[idx]['recipes_id'][1:])):
        users_df.at[idx, 'result_given_1'] = True
    if(is_slice_in_list(users_df.loc[idx]['pred_given_2'], users_df.loc[idx]['recipes_id'][2:])):
        users_df.at[idx, 'result_given_2'] = True
    if(is_slice_in_list(users_df.loc[idx]['pred_but_at_all'], users_df.loc[idx]['recipes_id'][:-1])):
        users_df.at[idx, 'result_but_at_all'] = True

### Given 1

In [90]:
correct_given_1 = users_df['result_given_1'].value_counts()[1]
correct_given_1 / len(users_df)

### Given 2

In [91]:
correct_given_2 = users_df['result_given_2'].value_counts()[1]
correct_given_2 / len(users_df)

### But at all

In [92]:
correct_but_at_all = users_df['result_but_at_all'].value_counts()[1]
correct_but_at_all / len(users_df)

### Save recommend list

In [95]:
recipes_pred = recipes_df[['id', 'recipes_red']]

In [96]:
recipes_pred = recipes_pred.set_index('id')

In [97]:
# recipes_pred.to_json('recipes_red.json')